<a href="https://colab.research.google.com/github/TheGreatJanus/ML-project2-text-classification/blob/main/Keras_main_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [411]:
#From Tenserflow
from keras import Sequential
from keras.layers import Embedding, Dense, GlobalAveragePooling1D, Dropout
import tensorflow as tf
import keras
from keras import layers
from keras import losses

import pandas as pd
import gzip
import nltk
from nltk.corpus import stopwords
import warnings; warnings.simplefilter('ignore')
import os, codecs, string, random
import numpy as np
from numpy.random import seed as random_seed
from numpy.random import shuffle as random_shuffle
import matplotlib.pyplot as plt
%matplotlib inline  

seed = 42
random.seed(seed)
np.random.seed(seed)

#NLP libraries
import spacy, nltk, gensim, sklearn


#Scikit imports
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier


In [412]:
print(tf.__version__)

2.9.2


In [413]:
# import small train and test datasets
train_neg = [tweet[:-1] for tweet in open('/twitter-datasets/train_neg.txt').readlines()]
train_pos = [tweet[:-1] for tweet in open('/twitter-datasets/train_pos.txt').readlines()]
test_data = [tweet[:-1] for tweet in open('/twitter-datasets/test_data.txt').readlines()]

In [414]:
X, y = train_neg + train_pos, [0 for i in range(len(train_neg))]+[1 for i in range(len(train_pos))]
X_test, y_test = test_data, [0 for i in range(len(test_data))] 
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.1)

In [415]:
#X_test = np.array(X_test).reshape(-1)
#X_train = np.array(X_train).reshape(-1)
#X_eval = np.array(X_eval).reshape(-1)

#y_test = np.array(y_test).reshape(-1)
#y_train = np.array(y_train).reshape(-1)
#y_eval = np.array(y_eval).reshape(-1)


In [416]:
raw_train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
raw_eval_ds = tf.data.Dataset.from_tensor_slices((X_eval, y_eval))
raw_test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [417]:
#Shuffle and Batch
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

#train_dataset = raw_train_ds.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
#eval_dataset = raw_eval_ds.batch(BATCH_SIZE)

In [418]:
max_features = 94953
sequence_length = 100

vectorize_layer = layers.TextVectorization(max_tokens=max_features,
    standardize= 'lower_and_strip_punctuation',
    output_mode='int',
    output_sequence_length=sequence_length
)

In [419]:
# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [420]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [421]:
i=0
for element in raw_train_ds:
  print(element)
  i+=1
  if i==10 :break


(<tf.Tensor: shape=(), dtype=string, numpy=b'babe i have plans to go to the mall'>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b"zatarain's new orleans style white beans & rice , 7 - ounce boxes ( pack of 12 this easy-to-prepare dinner mix has ... <url>">, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'dont wanna hear that its really hot in portugal'>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b"i've been going to bed early lately , i like the extra sleep">, <tf.Tensor: shape=(), dtype=int32, numpy=1>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'the great ribs book ( paperback the authors of our popular hot series ( over 200,000 books in print ) team up to ... <url>'>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b"i have so much track dirt on my jeans from #fenway100 ... i think i shall retire these pants and 

In [422]:
print(" 3134 ---> ",vectorize_layer.get_vocabulary()[1500])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

 3134 --->  peace
Vocabulary size: 94953


In [423]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_eval_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [424]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

**Creating the model**

In [425]:
embedding_dim = 4

In [426]:
model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1),
  layers.Flatten()])
model.summary()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_24 (Embedding)    (None, None, 4)           379816    
                                                                 
 dropout_48 (Dropout)        (None, None, 4)           0         
                                                                 
 global_average_pooling1d_24  (None, 4)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dropout_49 (Dropout)        (None, 4)                 0         
                                                                 
 dense_24 (Dense)            (None, 1)                 5         
                                                                 
 flatten_1 (Flatten)         (None, 1)                 0         
                                                     

In [427]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [428]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/10


ValueError: ignored

In [ ]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
acc = history_dict['binary_accuracy']
val_acc = history_dict['val_binary_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.show()

In [ ]:
export_model = tf.keras.Sequential([
  vectorize_layer,
  model,
  layers.Activation('sigmoid')
])

export_model.compile(
    loss=losses.BinaryCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)

# Test it with `raw_test_ds`, which yields raw strings
loss, accuracy = export_model.evaluate(raw_test_ds)
print(accuracy)

In [ ]:
export_model.predict(examples)